In [ ]:
import wget

In [ ]:
#wget.download('http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz')

In [2]:
import pandas as pd
import gzip
import json

data = []
with gzip.open('All_Beauty_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()).get('reviewText'))

print(len(data))

print(data[0])

data = [x for x in data if x!=None]

print(len(data))

5269
As advertised. Reasonably priced
5264


В чём, собственно, проблема, о наличии которой можно подумать, увидев это высказывание -- в отзывы на конкретный продукт сам продукт можно и не упоминать

# 1. Как найти товары: способы

*(3 балла) Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?*

__1. Читать и размечать, как, например, именованные сущности. После определённого количества размеченного можно попробовать обучить модель.__
 <br>
Плюсы: точно; не надо быть лингвистом; можно отдать на Толоку<br>
Минусы: долго и трудозатртно; можеть быть дорого; может получиться далеко не так точно, как предполагалось, из-за несогласия разметчиков или неясности правил<br>
 <br>
__2. Сложить всё в ворд-ту-век, надеяться на то, что упоминания товаров окажутся там в одном кластере__<br>
 <br>
Плюсы: Если повезёт, можно сгруппировать упоминания одного товара/синонимы. Относительно быстро. Не надо привлекать людей<br>
Минусы: Упоминания товаров часто не однословны, нужно что-то с биграммами придумывать. Или с триграммами. <br>
 <br>
__3. Привлечь тяжёлую артиллерию, а именно млодели для извлечения именованных сущностей__<br>
 <br>
Плюсы: Есть вероятность, что получится неплохая точность. Можно расширять список за счёт применения новых и новых идей<br>
Минусы: сложно, получится, скорее всего, не идеально
 <br>

# Извлечение названий товаров

*(2 балла) Реализуйте один из предложенных вами способов.*

Ну что ж, попробуем. Посмотрим на наши данные!

In [ ]:
data[30:50]

Здесь выделяются три типа упоминаний. <br>
1. Никак не упоминаем продукт или называем его "it"
2. this NOUN, ADJ + NOUN
3. Собственно названия
 <br>
 
И что же мы будем с этим делать? Ну, давайте действовать в два этапа. Даже в три. Пункт первый -- постараться извлечь именованные сущности. Пункт второй -- извлечь всё, что this + NOUN. Дальше ADJ + NOUN, но там будет много мусора

### Достаём именованные сущности

Stanza очень хороша! Но при этом она работает достаточно долго. Поэтому давайте именно её запустим на небольшом датасете, подвыборке из нашего большого. Если всё закончится хорошо, можно будет запустить вторую тетрадку с тем же кодом -- но другими данными

In [ ]:
import stanza
from tqdm import tqdm
#stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
def stanza_result(text, nlp=nlp):
    doc = nlp(text)
    ent_texts = [ent.text for sent in doc.sentences for ent in sent.ents]
    ent_types = [ent.type for sent in doc.sentences for ent in sent.ents]
    #print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
    return ent_texts, ent_types

In [ ]:
ent_texts = [] 
ent_types = []
for el in tqdm(data):
    #print(i)
    a = stanza_result(el)
    ent_texts.extend(a[0])
    ent_types.extend(a[1])

In [ ]:
products = []
for i, el in enumerate(ent_types):
    if el == 'PRODUCT':
        products.append(i)
        print(ent_texts[i])

In [ ]:
len(products)

### Часть вторая

Во-первых, stanza достатчно хороша в доставании названий продуктов. Даже очень хороша. Временами она достаёт названия фирм или компаний -- но во многих случаях это довольно логично: многие упоминания выглядят, как "шампунь X". А нам такое не очень нравится, наверное, потому что мы, вероятно, хотим знать, какой именно это продукт. Поэтому давайте экспериментировать с морфологией и отзывами.<br>
 <br>
Можно сразу начать думать обо всех пунктах задания. Что нам, в общем-то, нужно? Нам нужны упоминания товаров. Желательно всех товаров, которые упоминаются в отзыве, а он там может быть не один. Чтобы такое добыть и, например, сформировать хорошие N-граммы с сущностями, по которым потом тоже можно искать, можно посмотреть на то, как наши упоминания распределяются по отзывам.

(1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)<br>
 <br>
(3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).<br>
 <br>
(1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров. Должны получиться примерно такие группы:<br>
 <br>
(2 балла): если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

In [6]:
import stanza
from tqdm import tqdm
#stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos,depparse,ner', use_gpu=False)

2021-12-06 21:33:50 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

2021-12-06 21:33:50 INFO: Use device: cpu
2021-12-06 21:33:50 INFO: Loading: tokenize
2021-12-06 21:33:50 INFO: Loading: pos
2021-12-06 21:33:50 INFO: Loading: lemma
2021-12-06 21:33:50 INFO: Loading: depparse
2021-12-06 21:33:50 INFO: Loading: ner
2021-12-06 21:33:50 INFO: Done loading processors!


In [7]:
def stanza_doc(text, nlp=nlp):
    doc = nlp(text)
    return doc

In [10]:
def stanza_result(doc):
    ent_texts = [ent.text for sent in doc.sentences for ent in sent.ents]
    ent_types = [ent.type for sent in doc.sentences for ent in sent.ents]
    ent_ngrams = []
    this_list_1 = []
    this_list_2 = []
    one_ngram = []
    for sent in doc.sentences:
        #list_tokens = list(token for token in sent.tokens if token.upos != 'PUNCT')
        list_tokens = list(token for token in sent.tokens)
        for i, token in enumerate(list_tokens):
            if token.words[0].lemma == 'this':
                if i + 1 < len(list_tokens):
                    this_list_1.append(list_tokens[i+1])
                    if i + 2 < len(list_tokens):
                        this_list_2.append(list_tokens[i+2])
        #print(sent.ents)
            if token.ner == 'B-ORG' or token.ner == 'B-PRODUCT':
                ent_ngrams.append(one_ngram)
                one_ngram = []
                if i > 0:
                    one_ngram.append(list_tokens[i-1])
                else:
                    one_ngram.append('START')
                one_ngram.append(token)
            elif token.ner == 'I-ORG' or token.ner == 'I-PRODUCT':
                one_ngram.append(token)
            elif token.ner == 'E-ORG' or token.ner == 'E-PRODUCT':
                one_ngram.append(token)
                if i + 1 < len(list_tokens):
                    one_ngram.append(list_tokens[i+1])
                else:
                    one_ngram.append('END')
            elif token.ner == 'S-ORG' or token.ner == 'S-PRODUCT':
                ent_ngrams.append(one_ngram)
                one_ngram = []
                if i > 0:
                    one_ngram.append(list_tokens[i-1])
                else:
                    one_ngram.append('START')
                one_ngram.append(token)
                if i + 1 < len(list_tokens):
                    one_ngram.append(list_tokens[i+1])
                else:
                    one_ngram.append('END')
        
    ent_ngrams.append(one_ngram)

            
    
    
    #print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
    return ent_texts, ent_types, ent_ngrams, this_list_1, this_list_2 

In [8]:
stanza_docs = []
for text in tqdm(data):
    stanza_docs.append(stanza_doc(text, nlp=nlp))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5264/5264 [36:36<00:00,  2.40it/s]


In [11]:
%%time
all_ent_texts = []
all_ent_types = []
all_ent_ngrams = []
all_this_list_1 = []
all_this_list_2 = []

for text in tqdm(data):
    ent_texts, ent_types, ent_ngrams, this_list_1, this_list_2 = stanza_result(stanza_doc(text, nlp=nlp))
    all_ent_texts.append(ent_texts)
    all_ent_types.append(ent_types)
    all_ent_ngrams.append(ent_ngrams)
    all_this_list_1.append(this_list_1)
    all_this_list_2.append(this_list_2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5264/5264 [35:25<00:00,  2.48it/s]

Wall time: 35min 25s


In [44]:
all_text = []
for one_doc in stanza_docs:
    for sent in one_doc.sentences:
        all_text.extend([word.text.lower() for word in sent.words])

## Считаем метрики

In [56]:
def count_freq(our_ngram, all_texts):
    stops = ['START', 'END']
    left_context = len([i for i in all_text if i == our_ngram[0] and not i in stops])
    right_context = len([i for i in  all_text if i == our_ngram[-1] and not i in stops])
    our_ent_freq = 0
    our_ent = our_ngram[1:-1]
    
    index_first = []
    index_second = []
    if len(our_ent) == 1:
        our_ent = our_ent[0]
        for word in all_text:
            if our_ent == word:
                our_ent_freq += 1
    
    else:
        for i, word in enumerate(all_texts):
            if word == our_ent[0]:
                index_first.append(i)
                
        for i, word in enumerate(our_ent[1:]):
            for el in index_first:
                if all_texts[el + i + 1] == word:
                    index_second.append(el)
            index_first = index_second
            index_second = []
        our_ent_freq = len(index_first)
        
    return left_context, right_context, our_ent_freq

In [57]:
def count_freq_ngram(our_ngram, all_texts):
    index_first = []
    index_second = []    
    for i, word in enumerate(all_texts):
        if word == our_ngram[0]:
            index_first.append(i)

    for i, word in enumerate(our_ngram[1:]):
        for el in index_first:
            if all_texts[el + i + 1] == word:
                index_second.append(el)
        index_first = index_second
        index_second = []
        
    return len(index_first)

In [41]:
all_word_ngrams = []

for one_text in all_ent_ngrams:
    for one_ngram in one_text:
        one_ngram_words = []
        for word in one_ngram:
            if word == 'START' or word == 'END':
                 one_ngram_words.append(word)
            else:
                one_ngram_words.append(word.text)
        if one_ngram_words != []:
            all_word_ngrams.append(one_ngram_words)

In [45]:
all_word_ngrams

[['but', 'the', 'Williams', 'Company', 'ran'],
 ['an', 'AQUA', 'VELVA', 'MAN', '"'],
 ['about', 'Aqua', 'Velva', 'is'],
 ['have', 'Ambergris', 'in'],
 ['in', 'Wal', '-', 'Mart', 'and'],
 ['START', 'Selenium', 'appeared'],
 ['at', 'Amazon', 'END'],
 ['an', 'Airbnb', '.'],
 ['at', 'a', 'Dave', '&', 'Buster', "'s", 'in'],
 ['am', 'Uber', 'pleased'],
 ['and', 'the', 'D&G', 'Lifht', 'Blue', 'is'],
 ['with', 'Uhuru', 'Naturals', 'Hair'],
 ['START', 'Google', 'the'],
 ['with', 'Northern', 'Fir', "'s", 'beard'],
 ['START', 'Eau', 'de', 'Hadrien', 'is'],
 ['but', 'the', 'Tangle', 'Taming', 'Conditioner', 'is'],
 ['tried', 'Kerastase', ','],
 [',', 'PHYTO', 'and'],
 ['wish', 'Redken', 'had'],
 ['with',
  'the',
  'Federal',
  'Trade',
  'Commission',
  'Guidelines',
  'on',
  'Testimony',
  'and',
  'Advertising',
  '.'],
 ['from', 'Sally', "'s", 'Beauty', 'Supply', '.'],
 ['START', 'The', 'PRELL', 'Conditioner', 'is'],
 ['through', 'Amazon', 'I'],
 ['by', 'Bain', 'De', 'Terre', 'END'],
 ['used'

In [18]:
import math

In [63]:
def count_metrics(our_ngram, all_text):
    freq_of_bigram = count_freq_ngram(our_ngram, all_text)
    left_context, center, right_context = count_freq(our_ngram, all_text)
    num_bigrams = len(all_text)/2
    #print(freq_of_bigram)
    if freq_of_bigram != 0 and left_context != 0 and right_context != 0 and center != 0:
        t_score_left = (freq_of_bigram - (left_context * center)/num_bigrams)/(freq_of_bigram**(1/2))
        t_score_right = (freq_of_bigram - (right_context * center)/num_bigrams)/(freq_of_bigram**(1/2))
        dice_left = (2*freq_of_bigram)/(left_context + center)
        dice_right = (2*freq_of_bigram)/(right_context + center)
        pmi_left = math.log2(freq_of_bigram / (left_context * center))
        pmi_right = math.log2(freq_of_bigram / (right_context * center))
        
    else:
        t_score_left, t_score_right, dice_left, dice_right, pmi_left, pmi_right = [0, 0, 0, 0, 0, 0]
    
    return t_score_left, t_score_right, dice_left, dice_right, pmi_left, pmi_right

In [64]:
t_scores_left, t_scores_right, dices_left, dices_right, pmis_left, pmis_right = [], [], [], [], [], []

for ngram in tqdm(all_word_ngrams):
    ngram = [word.lower() for word in ngram]
    t_score_left, t_score_right, dice_left, dice_right, pmi_left, pmi_right = count_metrics(ngram, all_text)
    t_scores_left.append(t_score_left)
    t_scores_right.append(t_score_right)
    dices_left.append(dice_left)
    dices_right.append(dice_right)
    pmis_left.append(pmi_left)
    pmis_right.append(pmi_right)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1454/1454 [01:03<00:00, 22.86it/s]


In [67]:
import numpy as np
a = np.argsort(t_scores_left)[:20]
for i in a:
    print(all_word_ngrams[i])

['the', 'Sanitizer', '.']
['the', 'Sonicare', '.']
['the', 'Sonicare', '.']
[',', 'Womens', '.']
[',', 'Womens', '.']
[',', 'Limonene', '.']
[',', 'Limonene', '.']
['to', 'Sensodyne', '.']
['to', 'Listerine', 'Total', 'Care', '.']
['the', 'Aquaphor', ',']
['the', 'Sensodyne', ',']
['the', 'Clinique', ',']
['the', 'Waterpik', '.']
['the', 'Waterpik', '.']
[',', 'Combe', 'Inc', '.']
[',', 'Combe', 'Inc', '.']
[',', 'Combe', 'Inc', '.']
['my', 'Sonicare', '.']
['a', 'WaterPik', '.']
['a', 'Sonicare', '.']


In [68]:
a = np.argsort(t_scores_right)[:20]
for i in a:
    print(all_word_ngrams[i])

[',', 'Fragrance', ',']
[',', 'Fragrance', ',']
['to', 'Amazon', '.']
['to', 'Amazon', '.']
['to', 'Amazon', '.']
['from', 'Amazon', ',']
['by', 'Pre', 'de', 'Provence', '.']
['with', 'Amazon', '.']
['from', 'Amazon', '.']
['from', 'Amazon', '.']
['at', 'Amazon', '.']
['from', 'Amazon', '.']
['at', 'Amazon', '.']
['at', 'Amazon', '.']
['with', 'Amazon', '.']
['with', 'Amazon', '.']
['with', 'Amazon', '.']
['at', 'Amazon', '.']
['with', 'Amazon', '.']
['at', 'Amazon', '.']


In [69]:
a = np.argsort(dices_left)[:20]
for i in a:
    print(all_word_ngrams[i])

['START', 'Dove', 'no']
['START', 'Crest', 'Crest', 'Pro']
['START', 'Eau', 'de', 'Hadrien', 'is']
['START', 'Amazon', 'fixed']
['START', 'Fragarant', 'is']
['START', 'Colgate', 'Enamel', 'Health', 'Anticavity', 'Fluoride', 'Sparkling', 'Fresh', 'Mint', 'Mouthwash', 'is']
['START', 'Henkel', 'quit']
['START', 'Dove', 'was']
['START', 'Colgate', 'claims']
['START', 'Henkel', 'quit']
['START', 'Colgate', 'Enamel', 'Health', 'Anticavity', 'Fluoride', 'Sparkling', 'Fresh', 'Mint', 'Mouthwash', 'is']
['START', 'Pre', 'de', 'Provence', 'Soap', ',']
['START', 'Pre', 'de', 'Provence', 'soaps']
['START', 'Weleda', 'has']
['START', 'Pre', 'de', 'Provence', 'has']
['START', 'Pre', 'de', 'Provence', 'does']
['START', 'Dove', 'no']
['START', 'Apple', 'Pear']
['START', 'The', 'Arcitec', '1090', 'looked']
['START', 'Apple', 'Pear']


In [70]:
a = np.argsort(dices_right)[:20]
for i in a:
    print(all_word_ngrams[i])

['START', 'Avalon', 'Organics', 'has']
['START', 'Henkel', 'quit']
['START', 'Crest', 'Crest', 'Pro']
['START', 'Urban', 'Spa', 'Get']
['START', 'The', 'Urban', 'Spa', 'Microfiber', 'Bath', 'Pillow', 'makes']
['START', 'Wish', 'Walgreens', 'kept']
['START', 'Perlier', 'makes']
['START', 'This', 'Lavender', 'Blossom', 'Water', 'is']
['START', 'Matrix', 'should']
['START', 'Fragarant', 'is']
['START', 'Crest', 'certainly']
['START', 'Pre', 'de', 'Provence', 'Shea', 'Butter', 'Enriched']
['START', 'The', 'Pre', 'De', 'Provence', 'Maison', 'French', 'Lavender', 'Blossom', 'Linen', 'END']
['START', 'Matrix', 'Biolage', 'Fortifying', 'conditioner']
['START', 'Amazon', 'fixed']
['START', 'Dove', 'no']
['START', 'Fekkai', 'does']
['START', 'Dove', 'no']
['in', 'Disney', 'World', 'END']
['by', 'Bain', 'De', 'Terre', 'END']


In [71]:
a = np.argsort(pmis_left)[:20]
for i in a:
    print(all_word_ngrams[i])

['the', 'Sanitizer', '.']
['to', 'Sensodyne', '.']
['to', 'Listerine', 'Total', 'Care', '.']
['the', 'Sensodyne', ',']
['the', 'Aquaphor', ',']
['the', 'Clinique', ',']
['the', 'Sonicare', '.']
['the', 'Sonicare', '.']
['my', 'Sonicare', '.']
[',', 'Tocopheryl', 'Acetate', ',']
[',', 'Cinnamomum', 'Cassia', 'Leaf', 'Oil', ',']
[',', 'Lavender', 'Oil', ',']
[',', 'Soybean', 'Oil', ',']
[',', 'Pectin', ',']
[',', 'Ceteareth', '-', '20', ',']
[',', 'Potassium', 'Sorbate', ',']
[',', 'Dehydroxanthan', 'Gum', ',']
[',', 'Orange', 'Oil', ',']
[',', 'Pogostemon', 'Cablin', 'Oil', ',']
[',', 'Bergamot', 'Fruit', 'Oil', ',']


In [72]:
a = np.argsort(pmis_right)[:20]
for i in a:
    print(all_word_ngrams[i])

['from', 'Amazon', ',']
['by', 'Pre', 'de', 'Provence', '.']
[',', 'Fragrance', ',']
[',', 'Fragrance', ',']
['to', 'Amazon', '.']
['to', 'Amazon', '.']
['to', 'Amazon', '.']
['from', 'Pre', 'de', 'Provence', 'and']
['with', 'Amazon', '.']
['from', 'Amazon', '.']
['at', 'Amazon', '.']
['with', 'Amazon', '.']
['at', 'Amazon', '.']
['at', 'Amazon', '.']
['with', 'Amazon', '.']
['with', 'Amazon', '.']
['at', 'Amazon', '.']
['from', 'Amazon', '.']
['from', 'Amazon', '.']
['with', 'Amazon', '.']


Метрика DICE с правым контекстом больше всех похожа на правду. А ещё стоило избавиться от всей пунктуации, перезапустив тетрадку, но я заметила, что это не та тетрадка, слишклм поздно

## Выводим тип продуктов

In [84]:
for word in ['shampoo', 'conditioner', 'soap', 'butter', 'oil']:
    print(word)
    print('____________________')
    for ngram in all_word_ngrams:
        ngram_new = [el.lower() for el in ngram]
        if word in ngram:
            print(ngram)

shampoo
____________________
['the', 'H20', 'shampoo']
['the', 'H20', 'shampoo']
['the', 'H20', 'shampoo']
['the', 'H20', 'shampoo']
['the', 'H20', 'shampoo']
['This', 'Volume', 'Conditioner', 'shampoo']
conditioner
____________________
['the', 'Matrix', 'conditioner']
['START', 'Matrix', 'Biolage', 'Fortifying', 'conditioner']
['the', 'Matrix', 'conditioner']
['START', 'Matrix', 'Biolage', 'Fortifying', 'conditioner']
['the', 'Matrix', 'conditioner']
['START', 'Matrix', 'Biolage', 'Fortifying', 'conditioner']
['the', 'Matrix', 'conditioner']
['START', 'Matrix', 'Biolage', 'Fortifying', 'conditioner']
['the', 'Matrix', 'conditioner']
['START', 'Matrix', 'Biolage', 'Fortifying', 'conditioner']
soap
____________________
['the', 'Sea', 'Salt', 'w', '/', 'Kelp', 'soap']
['the', 'Sea', 'Salt', 'w', '/', 'Kelp', 'soap']
['the', 'Sea', 'Salt', 'w', '/', 'Kelp', 'soap']
['the', 'Sea', 'Salt', 'w', '/', 'Kelp', 'soap']
['the', 'Sea', 'Salt', 'w', '/', 'Kelp', 'soap']
['the', 'Sea', 'Salt', 'w',

У нас полуичилось найти упоминания. Это хорошо. Объёмный препроцессинг надо бы перезапустить -- это плохо. Но тем не менее, мы посчитали метрики и мы нашли упоминания в текстах вместе с n-граммами. Это тоже хорошо. 